In [1]:
# Install the required package for YOLO11
%pip install ultralytics

  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.4 MB/s eta 0:00:00a 0:00:01
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO("yolo11n.pt")

# Export the model to TFLite format
model.export(format="coreml", device='mps', nms=True)  # creates 'yolo11n_float32.tflite'



Ultralytics 8.3.130 🚀 Python-3.10.17 torch-2.6.0 MPS (Apple M1)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)
requirements: Ultralytics requirement ['coremltools>=8.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.4 MB/s eta 0:00:00a 0:00:01

requirements: AutoUpdate success ✅ 6.4s, installed 1 package: ['coremltools>=8.0']
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect




CoreML: starting export with coremltools 8.3.0...


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 65.00 passes/s]


CoreML Pipeline: starting pipeline with coremltools 8.3.0...
CoreML Pipeline: pipeline success
CoreML: export success ✅ 29.0s, saved as 'yolo11n.mlpackage' (5.2 MB)

Export complete (29.9s)
Results saved to /Users/kimwash/Projects/where2throw
Predict:         yolo predict task=detect model=yolo11n.mlpackage imgsz=640  
Validate:        yolo val task=detect model=yolo11n.mlpackage imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


'yolo11n.mlpackage'

In [9]:
# Load the exported TFLite model
tflite_model = YOLO("yolo11n_saved_model/yolo11n_float32.tflite")

# Run inference
results = tflite_model("IMG_0383.jpeg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolo11n_saved_model/yolo11n_float32.tflite for TensorFlow Lite inference...

image 1/1 /Users/kimwash/Projects/where2throw/IMG_0383.jpeg: 640x640 1 laptop, 1 cell phone, 137.0ms
Speed: 32.3ms preprocess, 137.0ms inference, 23.6ms postprocess per image at shape (1, 3, 640, 640)


In [10]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

In [2]:
%pip install numpy opencv-python tflite-runtime

ERROR: Could not find a version that satisfies the requirement tflite-runtime (from versions: none)
ERROR: No matching distribution found for tflite-runtime
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2

# TFLite 모델 로드
interpreter = tf.lite.Interpreter(model_path='./yolo11n_saved_model/yolo11n_float32.tflite')
interpreter.allocate_tensors()

# 입력 및 출력 텐서 정보 가져오기
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 입력 이미지 전처리
image = Image.open('bus.jpg').convert('RGB')
input_shape = input_details[0]['shape']
image_resized = image.resize((input_shape[2], input_shape[1]))
input_data = np.expand_dims(np.array(image_resized, dtype=np.float32), axis=0)

# 추론 실행
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# 결과 가져오기
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data.shape)


(1, 84, 8400)


/Users/kimwash/miniforge3/envs/waste-classifier/lib/python3.10/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:

import numpy as np
import cv2

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def process_output(output_data, conf_threshold=0.5, iou_threshold=0.4, input_size=640):
    # (1, 84, 8400) → (8400, 84)
    predictions = np.transpose(output_data[0])  # shape: (8400, 84)

    boxes = predictions[:, 0:4]  # cx, cy, w, h
    objectness = sigmoid(predictions[:, 4])
    class_scores = sigmoid(predictions[:, 5:])  # shape: (8400, 80)

    
    scores = objectness[:, np.newaxis] * class_scores
    class_ids = np.argmax(scores, axis=1)
    confidences = np.max(scores, axis=1)
    

    # confidence 필터링
    mask = confidences > conf_threshold
    boxes = boxes[mask]
    confidences = confidences[mask]
    class_ids = class_ids[mask]

    # 좌표 변환 (cx, cy, w, h) → (x1, y1, x2, y2)
    boxes_xywh = boxes * input_size
    boxes_xyxy = np.zeros_like(boxes_xywh)
    boxes_xyxy[:, 0] = boxes_xywh[:, 0] - boxes_xywh[:, 2] / 2  # x1
    boxes_xyxy[:, 1] = boxes_xywh[:, 1] - boxes_xywh[:, 3] / 2  # y1
    boxes_xyxy[:, 2] = boxes_xywh[:, 0] + boxes_xywh[:, 2] / 2  # x2
    boxes_xyxy[:, 3] = boxes_xywh[:, 1] + boxes_xywh[:, 3] / 2  # y2

    # NMS 수행
    indices = cv2.dnn.NMSBoxes(
        bboxes=boxes_xyxy.tolist(),
        scores=confidences.tolist(),
        score_threshold=conf_threshold,
        nms_threshold=iou_threshold
    )

    results = []
    if len(indices) > 0:
        for idx in indices.flatten():  # 1차원 배열로 확실히 만들어줌
            if confidences[idx] > 0.35:
                results.append({
                    "box": boxes_xyxy[idx],
                    "confidence": float(confidences[idx]),
                    "class_id": int(class_ids[idx])
                })


    return results



In [11]:
# 추론 결과 얻기
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
interpreter.allocate_tensors()
output_details = interpreter.get_output_details()
print(output_details[0]['shape'])  # 이게 진짜 shape임

# # 후처리 수행
# results = process_output(output_data, conf_threshold=0.1, iou_threshold=0.5, input_size=640)
# # 결과 출력
# for det in results:
#     box = det['box']  # [x1, y1, x2, y2]
#     cls = det['class_id']
#     conf = det['confidence']
#     print(f"Class {cls} | Confidence {conf:.2f} | Box: {box}")


[   1   84 8400]


In [4]:
import cv2

def draw_detections(image, results, class_names=None):
    """
    image: BGR 이미지 (cv2.imread로 불러온 것)
    results: process_output()에서 나온 리스트
    class_names: 선택적 클래스 이름 리스트 (COCO 클래스 등)
    """
    for det in results:
        x1, y1, x2, y2 = map(int, det['box'])
        cls = det['class_id']
        conf = det['confidence']

        label = f"{class_names[cls] if class_names else cls}: {conf:.2f}"
        
        # 바운딩 박스
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # 텍스트 배경
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image, (x1, y1 - 20), (x1 + w, y1), (0, 255, 0), -1)
        cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, (0, 0, 0), 1, cv2.LINE_AA)
    
    return image


In [5]:
original_image = Image.open("test.jpg").convert("RGB")
original_size = original_image.size  # (width, height)

image_resized = original_image.resize((640, 640))
input_data = np.expand_dims(np.array(image_resized, dtype=np.float32), axis=0)
def resize_boxes_to_original(results, original_size, input_size=640):
    """
    input_size: 모델 입력 크기 (기본 640)
    original_size: (width, height)
    """
    ow, oh = original_size
    scale_w = ow / input_size
    scale_h = oh / input_size

    for det in results:
        x1, y1, x2, y2 = det['box']
        det['box'] = [
            x1 * scale_w,
            y1 * scale_h,
            x2 * scale_w,
            y2 * scale_h
        ]
    return results

results = process_output(output_data, conf_threshold=0.1, iou_threshold=0.5, input_size=640)
results = resize_boxes_to_original(results, original_size=(original_image.width, original_image.height))
print(results)
# 이제 원본 이미지 위에 그릴 수 있음
image_bgr = cv2.cvtColor(np.array(original_image), cv2.COLOR_RGB2BGR)
output_image = draw_detections(image_bgr.copy(), results)
# 결과 보기
cv2.imshow("Detections", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[{'box': [np.float32(828.0538), np.float32(856.79724), np.float32(882.94025), np.float32(856.8211)], 'confidence': 0.3896104693412781, 'class_id': 31}, {'box': [np.float32(1412.9553), np.float32(2293.2), np.float32(1750.9581), np.float32(2368.1982)], 'confidence': 0.36552929878234863, 'class_id': 66}, {'box': [np.float32(1397.1947), np.float32(1083.5284), np.float32(1878.2823), np.float32(1086.8567)], 'confidence': 0.36552926898002625, 'class_id': 66}, {'box': [np.float32(2171.0056), np.float32(1555.3418), np.float32(2620.5298), np.float32(1738.0841)], 'confidence': 0.36552688479423523, 'class_id': 66}, {'box': [np.float32(1821.9312), np.float32(2646.0), np.float32(1990.9468), np.float32(2646.307)], 'confidence': 0.3655221164226532, 'class_id': 66}, {'box': [np.float32(544.38806), np.float32(981.4581), np.float32(678.51306), np.float32(1033.1919)], 'confidence': 0.3655214011669159, 'class_id': 66}, {'box': [np.float32(193.11188), np.float32(1184.195), np.float32(298.2326), np.float32(1

UsageError: Line magic function `%yolo` not found.
